In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200727.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],Rank your movie choices. You don't have to do a full ranking. [Eigth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eight Choice],What theme should we explore next week? [Nineth Choice]
0,7/22/2020 19:57:36,A Fistful of Dollars,The Italian Job,The Beyond,The Great Beauty,Benvenuti al Sud,Big Deal on Madonna Street,Nights of Cabiria,Porco Rosso,CONSPIRACY THEORIES,ALIENS,BANNED BY FASCISTS,DIRTY COPS!,MAYBE WE SHOULDN'T DO A WAR,MONOSYLLABIC TITLE,CRITICS HATED IT,DISGUISES,No Theme/Free-for-all
1,7/22/2020 19:58:30,The Beyond,Nights of Cabiria,The Great Beauty,Big Deal on Madonna Street,Benvenuti al Sud,The Italian Job,Porco Rosso,A Fistful of Dollars,MONOSYLLABIC TITLE,BANNED BY FASCISTS,ALIENS,CONSPIRACY THEORIES,DISGUISES,DIRTY COPS!,CRITICS HATED IT,NaN,NaN
2,7/22/2020 20:12:48,Nights of Cabiria,Big Deal on Madonna Street,Porco Rosso,The Beyond,Benvenuti al Sud,NaN,NaN,NaN,BANNED BY FASCISTS,MAYBE WE SHOULDN'T DO A WAR,No Theme/Free-for-all,CRITICS HATED IT,MONOSYLLABIC TITLE,ALIENS,CONSPIRACY THEORIES,DISGUISES,DIRTY COPS!
3,7/22/2020 20:17:04,Benvenuti al Sud,Porco Rosso,The Italian Job,The Beyond,A Fistful of Dollars,Big Deal on Madonna Street,The Great Beauty,Nights of Cabiria,ALIENS,BANNED BY FASCISTS,MONOSYLLABIC TITLE,CRITICS HATED IT,DISGUISES,CONSPIRACY THEORIES,DIRTY COPS!,MAYBE WE SHOULDN'T DO A WAR,No Theme/Free-for-all
4,7/22/2020 20:21:11,The Italian Job,Benvenuti al Sud,Porco Rosso,The Great Beauty,The Beyond,Big Deal on Madonna Street,A Fistful of Dollars,Nights of Cabiria,BANNED BY FASCISTS,CRITICS HATED IT,DISGUISES,ALIENS,DIRTY COPS!,MAYBE WE SHOULDN'T DO A WAR,MONOSYLLABIC TITLE,CONSPIRACY THEORIES,No Theme/Free-for-all
5,7/23/2020 0:33:11,Porco Rosso,The Italian Job,NaN,NaN,NaN,NaN,NaN,NaN,ALIENS,MONOSYLLABIC TITLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7/26/2020 16:46:01,The Great Beauty,Benvenuti al Sud,Nights of Cabiria,The Beyond,A Fistful of Dollars,Big Deal on Madonna Street,The Italian Job,Porco Rosso,MONOSYLLABIC TITLE,CONSPIRACY THEORIES,ALIENS,DISGUISES,CRITICS HATED IT,BANNED BY FASCISTS,DIRTY COPS!,MAYBE WE SHOULDN'T DO A WAR,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['A Fistful of Dollars', 'The Italian Job', 'The Beyond',
       'The Great Beauty', 'Benvenuti al Sud',
       'Big Deal on Madonna Street', 'Nights of Cabiria', 'Porco Rosso'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'A Fistful of Dollars': <Candidate('A Fistful of Dollars')>,
 'The Italian Job': <Candidate('The Italian Job')>,
 'The Beyond': <Candidate('The Beyond')>,
 'The Great Beauty': <Candidate('The Great Beauty')>,
 'Benvenuti al Sud': <Candidate('Benvenuti al Sud')>,
 'Big Deal on Madonna Street': <Candidate('Big Deal on Madonna Street')>,
 'Nights of Cabiria': <Candidate('Nights of Cabiria')>,
 'Porco Rosso': <Candidate('Porco Rosso')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(A Fistful of Dollars, The Italian Job, The Beyond, The Great Beauty, Benvenuti al Sud, Big Deal on Madonna Street, Nights of Cabiria, Porco Rosso)>,
 <Ballot(The Beyond, Nights of Cabiria, The Great Beauty, Big Deal on Madonna Street, Benvenuti al Sud, The Italian Job, Porco Rosso, A Fistful of Dollars)>,
 <Ballot(Nights of Cabiria, Big Deal on Madonna Street, Porco Rosso, The Beyond, Benvenuti al Sud)>,
 <Ballot(Benvenuti al Sud, Porco Rosso, The Italian Job, The Beyond, A Fistful of Dollars, Big Deal on Madonna Street, The Great Beauty, Nights of Cabiria)>,
 <Ballot(The Italian Job, Benvenuti al Sud, Porco Rosso, The Great Beauty, The Beyond, Big Deal on Madonna Street, A Fistful of Dollars, Nights of Cabiria)>,
 <Ballot(Porco Rosso, The Italian Job)>,
 <Ballot(The Great Beauty, Benvenuti al Sud, Nights of Cabiria, The Beyond, A Fistful of Dollars, Big Deal on Madonna Street, The Italian Job, Porco Rosso)>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                     Votes  Status
--------------------------  -------  --------
The Italian Job                   1  Hopeful
Benvenuti al Sud                  1  Hopeful
Porco Rosso                       1  Hopeful
Nights of Cabiria                 1  Hopeful
The Beyond                        1  Hopeful
The Great Beauty                  1  Hopeful
A Fistful of Dollars              1  Rejected
Big Deal on Madonna Street        0  Rejected

ROUND 2
Candidate                     Votes  Status
--------------------------  -------  --------
The Italian Job                   2  Hopeful
Porco Rosso                       1  Hopeful
Benvenuti al Sud                  1  Hopeful
The Beyond                        1  Hopeful
Nights of Cabiria                 1  Hopeful
The Great Beauty                  1  Rejected
A Fistful of Dollars              0  Rejected
Big Deal on Madonna Street        0  Rejected

ROUND 3
Candidate                     Votes  Status
-------------------------

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
BANNED BY FASCISTS                 2  Hopeful
ALIENS                             2  Hopeful
MONOSYLLABIC TITLE                 2  Hopeful
CONSPIRACY THEORIES                1  Rejected
CRITICS HATED IT                   0  Rejected
MAYBE WE SHOULDN'T DO A WAR        0  Rejected
DISGUISES                          0  Rejected
No Theme/Free-for-all              0  Rejected
DIRTY COPS!                        0  Rejected

ROUND 2
Candidate                      Votes  Status
---------------------------  -------  --------
ALIENS                             3  Hopeful
BANNED BY FASCISTS                 2  Hopeful
MONOSYLLABIC TITLE                 2  Rejected
CONSPIRACY THEORIES                0  Rejected
CRITICS HATED IT                   0  Rejected
MAYBE WE SHOULDN'T DO A WAR        0  Rejected
DISGUISES                          0  Rejected
No Theme/Free-for-all              0  Rejected
DIRTY